In [1]:
import pandas as pd
import geopandas as gpd
import folium
import pandas as pd
from branca.colormap import linear
from DadosAbertosBrasil import ibge

/Users/juliasbardelatti/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
novos_nomes = ['regiao', 'codigo_uf', 'uf', 'codigo_ibge', 'municipio', 
               'qtd_magreza_acentuada', 'perc_magreza_acentuada',
               'qtd_magreza', 'perc_magreza',
               'qtd_eutrofia', 'perc_eutrofia',
               'qtd_risco_sobrepeso', 'perc_risco_sobrepeso',
               'qtd_sobrepeso', 'perc_sobrepeso',
               'qtd_obesidade', 'perc_obesidade', 'total']

In [3]:
df = pd.read_csv("2024.csv", sep=";",names=novos_nomes)

In [4]:
df_estado = df[df['regiao'].str.startswith(('TOTAL ESTADO'), na=False)]
df_estado['regiao'].unique()

array(['TOTAL ESTADO DISTRITO FEDERAL', 'TOTAL ESTADO GOIÁS',
       'TOTAL ESTADO MATO GROSSO DO SUL', 'TOTAL ESTADO MATO GROSSO',
       'TOTAL ESTADO ALAGOAS', 'TOTAL ESTADO BAHIA', 'TOTAL ESTADO CEARÁ',
       'TOTAL ESTADO MARANHÃO', 'TOTAL ESTADO PARAÍBA',
       'TOTAL ESTADO PERNAMBUCO', 'TOTAL ESTADO PIAUÍ',
       'TOTAL ESTADO RIO GRANDE DO NORTE', 'TOTAL ESTADO SERGIPE',
       'TOTAL ESTADO ACRE', 'TOTAL ESTADO AMAZONAS', 'TOTAL ESTADO AMAPÁ',
       'TOTAL ESTADO PARÁ', 'TOTAL ESTADO RONDÔNIA',
       'TOTAL ESTADO RORAIMA', 'TOTAL ESTADO TOCANTINS',
       'TOTAL ESTADO ESPÍRITO SANTO', 'TOTAL ESTADO MINAS GERAIS',
       'TOTAL ESTADO RIO DE JANEIRO', 'TOTAL ESTADO SÃO PAULO',
       'TOTAL ESTADO PARANÁ', 'TOTAL ESTADO RIO GRANDE DO SUL',
       'TOTAL ESTADO SANTA CATARINA'], dtype=object)

In [5]:
df_estado['NOME'] = df_estado['regiao'].str.replace('TOTAL ESTADO ', '').str.strip()


/var/folders/gy/cxx2_pv95ys52g0qvnjgcsqm0000gn/T/ipykernel_48087/3520176744.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estado['NOME'] = df_estado['regiao'].str.replace('TOTAL ESTADO ', '').str.strip()


In [8]:
base_codigos = pd.read_csv("estados.csv")
base_codigos['NOME'] = base_codigos['NOME'].str.upper()
resultado = df_estado.merge(base_codigos, on='NOME', how='left')

In [9]:
colunas_para_converter = ['perc_magreza_acentuada', 'perc_magreza', 'perc_eutrofia', 'perc_risco_sobrepeso', 'perc_sobrepeso', 'perc_obesidade'] 

for coluna in colunas_para_converter:
    resultado[coluna] = resultado[coluna].str.replace('%', '', regex=False) 
    resultado[coluna] = pd.to_numeric(resultado[coluna], errors='coerce') 
    resultado[coluna] = resultado[coluna] / 100 


In [10]:
resultado['excesso_de_peso'] = (resultado['perc_sobrepeso'] + resultado['perc_obesidade']) 

In [11]:
geojson_path = ibge.malha(
         nivel = 'paises',
         localidade = 'BR',
         divisoes = 'uf',
         formato = 'geojson'
     )

In [13]:
mapa = folium.Map(location=[-14.2350, -51.9253], zoom_start=4, tiles=None)
colormap = linear.YlOrRd_09.scale(resultado["excesso_de_peso"].min(), resultado["excesso_de_peso"].max())
colormap.caption = 'Taxa mortalidade câncer colo do útero (por 100.000 mulheres)'
folium.Choropleth(
    geo_data=geojson_path,
    data=resultado,
    columns=["COD", "excesso_de_peso"],
    key_on="feature.properties.codarea",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Taxa mortalidade câncer colo do útero (por 100.000 mulheres)",
    nan_fill_color="white",
    nan_fill_opacity=0.3
).add_to(mapa)

if isinstance(geojson_path, dict):
    gdf = gpd.GeoDataFrame.from_features(geojson_path["features"])
else:
    gdf = gpd.read_file(geojson_path)  

gdf["codarea"] = gdf["codarea"].astype(int)

for _, row in gdf.iterrows():
    estado_codigo = row["codarea"]
    centroide = row.geometry.centroid  
    excesso_de_peso = resultado.loc[resultado["COD"] == estado_codigo, "excesso_de_peso"]
    
    if not excesso_de_peso.empty:
        # Converter para percentual e arredondar sem casas decimais
        excesso_de_peso = round(excesso_de_peso.values[0] * 100, 0)  

        folium.Marker(
    location=[centroide.y, centroide.x],
    icon=folium.DivIcon(html=f"""
        <div style="font-size: 12px; font-weight: bold; color: black; 
                    background-color: rgba(255, 255, 255, 0.8); 
                    padding: 5px 10px; border-radius: 8px; 
                    min-width: 50px; text-align: center;">
            {int(excesso_de_peso)}%
        </div>
    """)
).add_to(mapa)
        
        # Adiciona um título ao mapa
titulo_html = """
<div style="
    position: fixed; 
    top: 10px; left: 50%; 
    transform: translateX(-50%);
    z-index: 9999; 
    background-color: white; 
    padding: 10px; 
    border-radius: 8px;
    font-size: 18px; 
    font-weight: bold;
    box-shadow: 0 0 5px rgba(0,0,0,0.2);
">
    Mapa do percentual de crianças de 5 até 10 anos com excesso de peso (2024)
</div>
"""
mapa.get_root().html.add_child(folium.Element(titulo_html))

colormap.add_to(mapa)
mapa.get_root().html.add_child(folium.Element("""
    <style>
        html, body {
            background-color: white !important;
            height: 100%;
            margin: 0;
        }
        .folium-map { background: white; }
        #map { height: 100%; }
    </style>
"""))

mapa.save("teste.html")